In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import numpy as np

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((64, 64)),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

batch_size = 512

cifarset = CIFAR10(root='./data', train=True, download=True, transform=transform)

trainset, valid_set = random_split(cifarset, [40000, 10000])
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=4)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv2 = nn.Conv2d(64, 64, 5)
        
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 128, 3)
        
        self.conv5 = nn.Conv2d(128, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.aap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        
        x = self.aap(x)
        x = torch.flatten(x, 1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (aap): AdaptiveAvgPool2d(output_size=1)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)
# scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

In [11]:
train_loss_history, train_acc_history = [], []
valid_loss_history, valid_acc_history = [], []

In [12]:
epochs = 50

for epoch in range(epochs):   # 데이터셋을 수차례 반복합니다.
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    train_samples = 0
    valid_samples = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
#         scheduler.step()

        _, preds = torch.max(outputs, 1)
        train_loss += loss.item()
        train_acc += torch.sum(preds == labels.data)
        train_samples += len(inputs)
    
    else:
        # 훈련팔 필요가 없으므로 메모리 절약
        with torch.no_grad():
            for valid_input, valid_label in valid_loader:
                valid_input, valid_label = valid_input.to(device), valid_label.to(device)
                valid_outputs = net(valid_input)
                valid_loss = criterion(valid_outputs, valid_label)

                _, valid_preds = torch.max(valid_outputs, 1)
                valid_loss += valid_loss.item()
                valid_acc += torch.sum(valid_preds == valid_label.data)
                valid_samples += len(valid_input)
                
    epoch_loss = train_loss / len(trainloader)
    epoch_acc = train_acc.float() / train_samples * 100
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_acc)

    valid_epoch_loss = valid_loss * 10 / len(valid_loader)
    valid_epoch_acc = valid_acc.float() / valid_samples * 100
    valid_loss_history.append(valid_epoch_loss)
    valid_acc_history.append(valid_epoch_acc)

#     if (epoch + 1) % 5 == 0:
    print(f"epoch: {epoch + 1} || tl: {epoch_loss:.3f}, vl: {valid_epoch_loss:.3f} | ta: {epoch_acc:.3f}, va: {valid_epoch_acc:.3f}")

print('Finished Training')

epoch: 1 || tl: 2.257, vl: 2.162 | ta: 13.845, va: 17.220
epoch: 2 || tl: 2.118, vl: 2.020 | ta: 18.510, va: 20.240
epoch: 3 || tl: 2.022, vl: 2.008 | ta: 21.805, va: 23.050
epoch: 4 || tl: 1.952, vl: 1.982 | ta: 24.347, va: 25.300
epoch: 5 || tl: 1.901, vl: 1.833 | ta: 26.500, va: 27.780
epoch: 6 || tl: 1.867, vl: 1.893 | ta: 28.380, va: 28.790
epoch: 7 || tl: 1.836, vl: 1.795 | ta: 29.565, va: 30.950
epoch: 8 || tl: 1.816, vl: 1.801 | ta: 30.447, va: 30.760
epoch: 9 || tl: 1.802, vl: 1.794 | ta: 31.102, va: 31.710
epoch: 10 || tl: 1.779, vl: 1.754 | ta: 31.920, va: 33.050
epoch: 11 || tl: 1.760, vl: 1.743 | ta: 32.947, va: 32.920
epoch: 12 || tl: 1.751, vl: 1.803 | ta: 33.202, va: 34.150
epoch: 13 || tl: 1.741, vl: 1.700 | ta: 33.590, va: 34.820
epoch: 14 || tl: 1.726, vl: 1.762 | ta: 34.685, va: 35.260
epoch: 15 || tl: 1.715, vl: 1.685 | ta: 35.117, va: 34.660
epoch: 16 || tl: 1.702, vl: 1.697 | ta: 35.767, va: 36.070
epoch: 17 || tl: 1.684, vl: 1.608 | ta: 36.240, va: 36.390
epoch:

KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,5))
# plt.subplot(1, 2, 1)  
# plt.title("Training and Validation Loss")
# plt.plot(valid_loss_history,label="val")
# plt.plot(train_loss_history,label="train")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.legend()

# plt.subplot(1, 2, 2) 
plt.title("Training and Validation Acc")
plt.plot(valid_acc_history,label="val")
plt.plot(train_acc_history,label="train")
plt.xlabel("Epoch")
plt.ylabel("Acc")
plt.legend()
plt.show()

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# net = Net()
# net.load_state_dict(torch.load(PATH))
# net.to(device)